## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [11]:
bike_df = pd.read_csv('bikeshare.csv')

print(len(bike_df))

print(bike_df.head())

10886
              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [12]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [29]:
datetime=bike_df['datetime'].apply(pd.to_datetime)
datetime

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: datetime, Length: 10886, dtype: datetime64[ns]

- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

In [30]:
bike_df['datetime-year']=datetime.apply(lambda x: x.year)
bike_df['datetime-month']=datetime.apply(lambda x: x.month)
bike_df['datetime-day']=datetime.apply(lambda x: x.day)
bike_df['datetime-hour']=datetime.apply(lambda x: x.hour)


- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [35]:
bike_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,datetime-year,datetime-month,datetime-day,datetime-hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2


* datatime, causal, registered 칼럼을 삭제 하시오.(casual + registered = count이므로 따로 필요하지 않습니다.) 

In [40]:
bike_df.drop(['datetime','casual','registered'],axis=1, inplace=True)


In [41]:
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,datetime-year,datetime-month,datetime-day,datetime-hour
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4


## 2. 선형 회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. (종속변수 = count)

In [43]:
X_features=bike_df.drop(labels='count',axis=1).values
y_target=bike_df['count'].values

### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [47]:
X_features_ohe=pd.get_dummies(bike_df, columns=['datetime-year'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['datetime-month'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['datetime-day'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['datetime-hour'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['holiday'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['workingday'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['season'])

X_features_ohe=pd.get_dummies(X_features_ohe, columns=['weather'])

In [48]:
X_features_ohe.head()

,temp,atemp,humidity,windspeed,count,datetime-year_2011,datetime-year_2012,datetime-month_1,datetime-month_2,datetime-month_3,...,workingday_0,workingday_1,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4
0,9.84,14.395,81,0.0,16,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,0
1,9.02,13.635,80,0.0,40,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,0
2,9.02,13.635,80,0.0,32,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,0
3,9.84,14.395,75,0.0,13,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,0
4,9.84,14.395,75,0.0,1,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,0


### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0)

### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [96]:
model=LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [97]:
pred = model.predict(X_test)

In [98]:
RMSE = np.sqrt(mean_squared_error(y_test,pred))
i = model.intercept_
c = model.coef_
print(RMSE)
print(i)
print(c)

140.89958867165447
-164052.55926997645
[-7.99779518 -5.90658565  3.09937437 -5.54016474  2.57985532  3.8271057
 -2.05628773  0.32833324 81.57458005 10.23394918  0.47379273  7.90419231]


#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [99]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X_train, y_train)


SGDRegressor(eta0=0.1, penalty=None)

In [101]:
print(sgd_reg.intercept_)

print(sgd_reg.coef_)

[-2.04687092e+09]
[-8.78538722e+12  3.80904903e+11 -4.84854189e+10 -3.56585568e+12
 -5.58213669e+13 -5.46998709e+13 -3.71749124e+13  2.27400845e+12
 -3.32177010e+12 -2.70806565e+13  1.71278140e+13 -3.47076918e+13]


#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [102]:
from sklearn.linear_model import Ridge, Lasso


In [103]:
ridge_reg=Ridge(alpha=0.1)
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred)))
print(ridge_reg.intercept_)
print(ridge_reg.coef_)

140.899612655731
-164043.81404880635
[-7.99598527 -5.90391576  3.09939923 -5.53979213  2.579876    3.82709881
 -2.05630338  0.32831974 81.57023226 10.23336989  0.47379729  7.90417675]


In [104]:
lasso_reg=Lasso(alpha=0.1)
lasso_reg.fit(X_train, y_train)
lasso_pred = lasso_reg.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,lasso_pred)))
print(lasso_reg.intercept_)
print(lasso_reg.coef_)

140.89862972310257
-163182.58916639184
[-6.35905308 -2.77794117  2.93199509 -5.2006848   2.66127106  3.75303067
 -2.06192368  0.31933133 81.14183089  9.71108029  0.47167525  7.89917515]


## 3. 로지스틱 회귀 분석 실습 _ 와인 종류 분류하기 : 와인의 화학 조성을 사용하여 와인의 종류를 예측하기 위한 데이터이다. 

In [105]:
# 와인 데이터를 불러오기 위한 코드입니다. 꼭 실행해주세요.
from sklearn import datasets

ㅇ 타겟 데이터 : 와인의 종류 0, 1, 2의 세가지 값

ㅇ 특징 데이터

* 알콜(Alcohol)

* 말산(Malic acid)

* 회분(Ash)

* 회분의 알칼리도(Alcalinity of ash)

* 마그네슘(Magnesium)

* 총 폴리페놀(Total phenols)

* 플라보노이드 폴리페놀(Flavanoids)

* 비 플라보노이드 폴리페놀(Nonflavanoid phenols)

* 프로안토시아닌(Proanthocyanins)

* 색상의 강도(Color intensity)

* 색상(Hue)

* 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)

* 프롤린(Proline)

####3-1. 와인 데이터 가져오기 

In [106]:
wine_data = datasets.load_wine()

더 자세한 데이터에 대한 설명을 보고싶으시다면 아래의 코드를 실행해주시면 됩니다. 

In [107]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

#### 3-2. 입력변수와 목표변수 설정하기
* 입력변수를 사용하기 편하게 pandas dataframe으로 변환해주세요.
  - 입력변수는 data (.data) 함수를 써서 가져오시면 됩니다.
  - 칼럼명 가져오기 : .feature_names 
* 목표변수를 사용하기 편하게 pandas dataframe으로 변환해주세요
  - 목표변수는 target (.target) 함수를 써서 가져요오시면 됩니다. 
  - 칼럼명은 'target'으로 지정해주세요

In [117]:
# 입력 변수를 사용하기 편하게 pandas dataframe으로 변환
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# 목표 변수를 사용하기 편하게 pandas dataframe으로 변환
y = pd.DataFrame(wine_data.target,columns=['target'])


#### 3-3. 훈련 데이터, 테스트 데이터 만들기
* train_test_split 함수를 사용해 테스트 셋의 비율이 0.2가 되도록 X_train, X_test, y_train, y_test 을 만들어주세요. random_state=5로 설정해주세요.
* 경고메세지를 없애도록 y_train = y_train.values.ravel() 를 실행해주세요

In [118]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

y_train= y_train.values.ravel()

#### 3-4. 모델훈련
* 로지스틱회귀 함수에서 옵션을 solver='saga', max_iter=7500으로 설정해주세요.
* 학습데이터로 모델을 학습시켜주세요

In [119]:
# 필요한 라이브러리 import
from sklearn.linear_model import LogisticRegression

log_reg=LogisticRegression(solver='saga',max_iter=7500)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=7500, solver='saga')

#### 3-5. 예측
* 테스트 데이터 셋으로 로지스틱 회귀 모델을 이용해 와인 데이터 분류를 예측해 결과를 y_test_predict 변수에 담아주세요
* score함수를 사용해 로지스틱 회귀 모델의 성능을 score 변수에 담아주세요
* y_test_predict와 score 결과를 확인해주세요.

In [123]:
# 로지스틱 회귀 모델를 이용해서 각 와인 데이터 분류를 예측함
y_test_predict=log_reg.predict(X_test)

# 로지스틱 회귀 모델의 성능 확인 (정확성 %를 리턴함)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_test_predict)

# 결과확인 
y_test_predict, score

(array([0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2]),
 0.75)

#### (선택) 3-6. GridSearchCV 복습

`LogisticRegression` 모델에 `param_grid`를 사용하여 GridSearchCV를 진행해주세요.
- cv = 5
- 추가적으로 코드를 작성하실 필요 없이, 골뱅이@@@ 부분의 코드를 완성해서 출력해주시면 됩니다. 

In [124]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

# param_grid (수정하셔도 됩니다)
params = {
    'penalty':['l1', 'l2', 'elasticnet'],        # l1 is Lasso, l2 is Ridge
    'C': np.linspace(0.00002,1,100)
}

# GridSearch
grid_search = GridSearchCV(log_reg, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

학습된 gridsearch 모델에 저장되어 있는 best parameter, best score 를 출력해주세요.

In [ ]:
grid_search.best_params_, grid_search.best_score_